In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from tensorflow.keras.models import Sequential


from tensorflow.keras.layers import LSTM, Dense, Dropout, RepeatVector, TimeDistributed, Input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras import layers, Sequential, regularizers

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [2]:
# The DS is multivariate time series data with 3 target variables to forecast.
# In preprocessing notebook, I realized that the distribution of three zones are different.
# AS CNN notebook, this DS is a bit tricky too. 

In [3]:
df = pd.read_csv('preprocessed_hourly.csv', parse_dates=["DateTime"], index_col="DateTime")
df = df.sort_index()
df.head()

,Temperature,Humidity,Wind_Speed,general_diffuse_flows,diffuse_flows,zone_one,zone_two,zone_three,hour_sin,hour_cos,month_sin,month_cos
DateTime,,,,,,,,,,,,
2017-01-01 00:00:00,6.196833,75.066667,0.081833,0.063500,0.098833,29197.974683,18026.747720,19252.048193,0.000000,1.000000,0.0,1.0
2017-01-01 01:00:00,5.548833,77.583333,0.082000,0.056833,0.112500,24657.215190,16078.419453,17042.891567,0.258819,0.965926,0.0,1.0
2017-01-01 02:00:00,5.054333,78.933333,0.082333,0.063000,0.129167,22083.037973,14330.699088,15676.144578,0.500000,0.866025,0.0,1.0
2017-01-01 03:00:00,5.004333,77.083333,0.082833,0.059833,0.141000,20811.139240,13219.452887,14883.855422,0.707107,0.707107,0.0,1.0
2017-01-01 04:00:00,5.097667,74.050000,0.082333,0.058000,0.122833,20475.949367,12921.580547,14317.108433,0.866025,0.500000,0.0,1.0


In [4]:
df.columns

Index(['Temperature', 'Humidity', 'Wind_Speed', 'general_diffuse_flows',
       'diffuse_flows', 'zone_one', 'zone_two', 'zone_three', 'hour_sin',
       'hour_cos', 'month_sin', 'month_cos'],
      dtype='object')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8736 entries, 2017-01-01 00:00:00 to 2017-12-30 23:00:00
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Temperature            8736 non-null   float64
 1   Humidity               8736 non-null   float64
 2   Wind_Speed             8736 non-null   float64
 3   general_diffuse_flows  8736 non-null   float64
 4   diffuse_flows          8736 non-null   float64
 5   zone_one               8736 non-null   float64
 6   zone_two               8736 non-null   float64
 7   zone_three             8736 non-null   float64
 8   hour_sin               8736 non-null   float64
 9   hour_cos               8736 non-null   float64
 10  month_sin              8736 non-null   float64
 11  month_cos              8736 non-null   float64
dtypes: float64(12)
memory usage: 887.2 KB


In [6]:
df.shape

(8736, 12)

In [7]:
# Splitting the DS into train, val, test
Past_hours = 48
Next_hours = 2
#v Creatinig the gap between train, val, test. for avoiding leakage.
Gap_hours= Past_hours + Next_hours

Total_hours = len(df)
# Calculating the split 
# 70% train, 15% val, 15% test
train_end = int(Total_hours * 0.80)
val_end= int(Total_hours *0.90)

# Creating train, val, test DS with gap hours
train_df =df.iloc[:train_end]
val_df= df.iloc[train_end + Gap_hours:val_end]
test_df= df.iloc[val_end+ Gap_hours:]

print("train:", train_df.shape,train_df.index.min(),"to", train_df.index.max())
print("val :",val_df.shape,val_df.index.min(),"to",val_df.index.max())
print("test :", test_df.shape,test_df.index.min(),"to", test_df.index.max())
# Source: https://github.com/scikit-learn/scikit-learn/pull/13204

train: (6988, 12) 2017-01-01 00:00:00 to 2017-10-19 03:00:00
val : (824, 12) 2017-10-21 06:00:00 to 2017-11-24 13:00:00
test : (824, 12) 2017-11-26 16:00:00 to 2017-12-30 23:00:00


In [8]:

Zones_columns =["zone_one","zone_two", "zone_three"]
Continue_columns= ["Temperature", "Humidity","Wind_Speed","general_diffuse_flows", "diffuse_flows"]
Time_columns =["hour_sin","hour_cos","month_sin", "month_cos"]

#Time_columns =["is_weekend", "hour_sin","hour_cos", "days_sin", "days_cos","month_sin", "month_cos"]

x_scaler= RobustScaler()          
y_scaler = RobustScaler()
X_train_continues = x_scaler.fit_transform(train_df[Continue_columns])
X_val_continues= x_scaler.transform(val_df[Continue_columns])
X_test_continues =x_scaler.transform(test_df[Continue_columns])


X_train_time = train_df[Time_columns].to_numpy()
X_val_time= val_df[Time_columns].to_numpy()
X_test_time= test_df[Time_columns].to_numpy()

X_train = np.hstack([X_train_continues, X_train_time])
X_val = np.hstack([X_val_continues, X_val_time])
X_test= np.hstack([X_test_continues, X_test_time])


y_train = y_scaler.fit_transform(train_df[Zones_columns])
y_val= y_scaler.transform(val_df[Zones_columns])
y_test= y_scaler.transform(test_df[Zones_columns])


print("X shapes:",X_train.shape, X_val.shape, X_test.shape)

X shapes: (6988, 9) (824, 9) (824, 9)


In [9]:
# To forecast the next 12 hours, I will need to create series windows as I did in CNN exercise.
# To learn patterns, I will use the past 48 hours of data for forecasting the next 12 hours
def windows_time(X, y, input_steps=48, output_steps=6):
    X_windows, y_windows = [], []
    for i in range(len(X)-input_steps -output_steps+1):
        # Creating windows for X and y
        # 48 hours of features and target variables
        Past_X = X[i:i+input_steps, :]
        Past_y = y[i:i+input_steps, :]
        # Combing the past X and past y
        X_windows.append(np.hstack([Past_X, Past_y]))
        y_windows.append(y[i+input_steps:i+input_steps+output_steps, :])
    return np.array(X_windows), np.array(y_windows)

X_train_win, y_train_win= windows_time(X_train, y_train,Past_hours, Next_hours)
X_val_win,y_val_win= windows_time(X_val,y_val,Past_hours, Next_hours)
X_test_win, y_test_win= windows_time(X_test,y_test,Past_hours, Next_hours)

print("Train windows:", X_train_win.shape, y_train_win.shape)
print("Val windows:", X_val_win.shape,y_val_win.shape)
print("Test windows:",X_test_win.shape, y_test_win.shape)

Train windows: (6939, 48, 12) (6939, 2, 3)
Val windows: (775, 48, 12) (775, 2, 3)
Test windows: (775, 48, 12) (775, 2, 3)


In [10]:
print("X_train_win shape:", X_train_win.shape) 
print("y_train_win shape:", y_train_win.shape)  

print("PAST_STEPS:", X_train_win.shape[1])
print("FUTURE_STEPS:", y_train_win.shape[1])
print("N_FEATURES:", X_train_win.shape[2])
print("N_TARGETS:", y_train_win.shape[2])


X_train_win shape: (6939, 48, 12)
y_train_win shape: (6939, 2, 3)
PAST_STEPS: 48
FUTURE_STEPS: 2
N_FEATURES: 12
N_TARGETS: 3


<h2><center>Creating LSTM for multi zone<h2>

In [11]:
zones =["zone_one","zone_two","zone_three"]
Past_hours= X_train_win.shape[1]   
Inpute_shape =X_train_win.shape[2]  
Future_hours = y_train_win.shape[1]   

# Creating dictionary for saving the zones separately.
y_train_dict = {
    "zone_one":y_train_win[..., 0:1],
    "zone_two":y_train_win[..., 1:2],
    "zone_three":y_train_win[..., 2:3]}

y_val_dict = {
    "zone_one":y_val_win[..., 0:1],
    "zone_two": y_val_win[..., 1:2],
    "zone_three":y_val_win[..., 2:3]}

# I got the code structure form Qwen Max3. 

In [12]:
inputs = layers.Input(shape=(Past_hours, Inpute_shape), name="past_hours")
# Encoders 
x = layers.LSTM(128, activation="tanh",kernel_regularizer=regularizers.l2(0.00005))(inputs)
x = layers.Dropout(0.1)(x)

x = layers.RepeatVector(Future_hours)(x)

x = layers.LSTM(128, activation="tanh", return_sequences=True, kernel_regularizer=regularizers.l2(0.00005))(x)
x = layers.Dropout(0.1)(x)


shared= layers.TimeDistributed(layers.Dense(64, activation="tanh"), name="shared_td")(x)
# Output layers for each zone
z1 = layers.TimeDistributed(layers.Dense(32, activation="relu"))(shared)
out_zone1 = layers.TimeDistributed(layers.Dense(1), name="zone_one")(z1)
z2 = layers.TimeDistributed(layers.Dense(32, activation="relu"))(shared)
out_zone2= layers.TimeDistributed(layers.Dense(1), name="zone_two")(z2)
z3 = layers.TimeDistributed(layers.Dense(32, activation="relu"))(shared)
out_zone3 = layers.TimeDistributed(layers.Dense(1), name="zone_three")(z3)

model = tf.keras.Model(inputs, outputs={"zone_one": out_zone1,"zone_two": out_zone2,"zone_three": out_zone3})


loss_weights = {"zone_one": 1, "zone_two": 1, "zone_three": 1.15}
losses = {"zone_one": tf.keras.losses.LogCosh(),"zone_two": tf.keras.losses.LogCosh(),"zone_three": tf.keras.losses.LogCosh()}


model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001, clipnorm=1),loss=losses,
    loss_weights=loss_weights, metrics={"zone_one":["mae"], "zone_two":["mae"], "zone_three":["mae"]})


model.summary()

# Sources:
# https://stackoverflow.com/questions/50530100/keras-lstm-multi-output-model-predict-two-features-time-series
# https://github.com/keras-team/keras/issues/13453
# https://www.kaggle.com/code/kcostya/lstm-models-for-multi-step-time-series-forecast
# For debugging, I used Qwen Max3.

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ past_hours          │ (None, 48, 12)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 128)       │     72,192 │ past_hours[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 128)       │          0 │ lstm[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ repeat_vector       │ (None, 2, 128)    │          0 │ dropout[0][0]     │
│ (RepeatVector)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ (None, 2, 128)    │    131,584 │ repeat_vector[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 2, 128)    │          0 │ lstm_1[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared_td           │ (None, 2, 64)     │      8,256 │ dropout_1[0][0]   │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed    │ (None, 2, 32)     │      2,080 │ shared_td[0][0]   │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_2  │ (None, 2, 32)     │      2,080 │ shared_td[0][0]   │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_1  │ (None, 2, 32)     │      2,080 │ shared_td[0][0]   │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zone_one            │ (None, 2, 1)      │         33 │ time_distributed… │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zone_three          │ (None, 2, 1)      │         33 │ time_distributed… │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zone_two            │ (None, 2, 1)      │         33 │ time_distributed… │
│ (TimeDistributed)   │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 218,371 (853.01 KB)

 Trainable params: 218,371 (853.01 KB)

 Non-trainable params: 0 (0.00 B)

In [13]:
callbacks = [
    ModelCheckpoint( "best_multizone.keras", monitor="val_loss", mode = 'min', save_best_only=True, verbose=1 ),
    EarlyStopping( monitor="val_loss", patience=5, min_delta=0.00001, restore_best_weights=True),
    ReduceLROnPlateau( monitor="val_loss", factor=0.5, patience=3,min_delta = 0.000001, min_lr=0.000001, cooldown=1,verbose=1),]

In [14]:
history = model.fit(
    X_train_win, y_train_dict, validation_data=(X_val_win, y_val_dict),
    epochs=200,batch_size=128, shuffle=False, callbacks=callbacks, verbose=1)

Epoch 1/200
54/55 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.3087 - zone_one_loss: 0.1032 - zone_one_mae: 0.3759 - zone_three_loss: 0.0761 - zone_three_mae: 0.3123 - zone_two_loss: 0.1073 - zone_two_mae: 0.3906
Epoch 1: val_loss improved from None to 0.36195, saving model to best_multizone.keras
55/55 ━━━━━━━━━━━━━━━━━━━━ 6s 42ms/step - loss: 0.3109 - zone_one_loss: 0.0814 - zone_one_mae: 0.3283 - zone_three_loss: 0.1057 - zone_three_mae: 0.3736 - zone_two_loss: 0.0972 - zone_two_mae: 0.3656 - val_loss: 0.3619 - val_zone_one_loss: 0.0582 - val_zone_one_mae: 0.2987 - val_zone_three_loss: 0.0821 - val_zone_three_mae: 0.3252 - val_zone_two_loss: 0.2111 - val_zone_two_mae: 0.5907 - learning_rate: 0.0010
Epoch 2/200
54/55 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.1783 - zone_one_loss: 0.0498 - zone_one_mae: 0.2590 - zone_three_loss: 0.0588 - zone_three_mae: 0.2625 - zone_two_loss: 0.0514 - zone_two_mae: 0.2583
Epoch 2: val_loss improved from 0.36195 to 0.26364, saving model to best_multi

In [15]:
best_model = tf.keras.models.load_model("best_multizone.keras")

In [16]:
# Predicting on the test DS
y_pred_dict = best_model.predict(X_test_win, batch_size=128)

# Reconstructing the outputs
y_pred_stacked = np.stack([
    y_pred_dict["zone_one"].squeeze(-1),
    y_pred_dict["zone_two"].squeeze(-1),
    y_pred_dict["zone_three"].squeeze(-1)], axis=-1)

# Inversing to original scalle
y_pred_original = y_scaler.inverse_transform(y_pred_stacked.reshape(-1, 3)).reshape(y_pred_stacked.shape)
y_test_original = y_scaler.inverse_transform(y_test_win.reshape(-1, 3)).reshape(y_test_win.shape)


zones= ['zone_one', 'zone_two','zone_three']
for i, zone in enumerate(zones):
    y_true_zone= y_test_original[:, :, i].reshape(-1)
    y_pred_zone= y_pred_original[:, :, i].reshape(-1)
    
    mse= mean_squared_error(y_true_zone, y_pred_zone)
    mae = mean_absolute_error(y_true_zone, y_pred_zone)
    rmse= np.sqrt(mse)
    r2 = r2_score(y_true_zone, y_pred_zone)
    mape=np.mean(np.abs((y_true_zone- y_pred_zone)/(y_true_zone + 0.00000001)))*100
    
    print(f"\n{zone.upper()}:")
    print(f"MSE:{mse:,.2f}")
    print(f"RMSE: {rmse:,.2f}")
    print(f"MAE:{mae:,.2f}")
    print(f"R²:{r2:.4f} ({r2*100:.2f}%)")
    print(f" MAPE:{mape:.2f}%")


mse_all= mean_squared_error(y_test_original.reshape(-1),y_pred_original.reshape(-1))
mae_all= mean_absolute_error(y_test_original.reshape(-1), y_pred_original.reshape(-1))
r2_all =r2_score(y_test_original.reshape(-1) ,y_pred_original.reshape(-1))
print(f"\nMSE:{mse_all:,.2f}")
print(f"RMSE:{np.sqrt(mse_all):,.2f}")
print(f"MAE:{mae_all:,.2f}")
print(f"R²:{r2_all:.4f} ({r2_all*100:.2f}%)")

# I got the above code structure form Qwen Max3.

7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step

ZONE_ONE:
MSE:10,842,475.72
RMSE: 3,292.79
MAE:2,974.29
R²:0.7134 (71.34%)
 MAPE:10.47%

ZONE_TWO:
MSE:6,529,183.74
RMSE: 2,555.23
MAE:2,057.79
R²:0.7988 (79.88%)
 MAPE:8.11%

ZONE_THREE:
MSE:4,763,191.91
RMSE: 2,182.47
MAE:2,006.45
R²:0.4203 (42.03%)
 MAPE:19.94%

MSE:7,378,283.79
RMSE:2,716.30
MAE:2,346.18
R²:0.9118 (91.18%)


In [17]:
"""
model = Sequential([
    layers.Input(shape=(PAST_STEPS, N_FEATURES)),

    layers.LSTM(128, activation="tanh", kernel_regularizer=l2(0.00002)),
    layers.Dropout(0.2),

    layers.RepeatVector(FUTURE_STEPS),
    layers.LSTM(128, activation="tanh", return_sequences=True, kernel_regularizer=l2(0.00002)),
    layers.Dropout(0.2),

    layers.TimeDistributed(layers.Dense(64, activation="tanh")),
    layers.TimeDistributed(layers.Dense(N_TARGETS))])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0008),
    loss=tf.keras.losses.Huber(delta=1.0), metrics=[tf.keras.metrics.MeanAbsoluteError(name="mae")])

model.summary()

"""

'\nmodel = Sequential([\n    layers.Input(shape=(PAST_STEPS, N_FEATURES)),\n\n    layers.LSTM(128, activation="tanh", kernel_regularizer=l2(0.00002)),\n    layers.Dropout(0.2),\n\n    layers.RepeatVector(FUTURE_STEPS),\n    layers.LSTM(128, activation="tanh", return_sequences=True, kernel_regularizer=l2(0.00002)),\n    layers.Dropout(0.2),\n\n    layers.TimeDistributed(layers.Dense(64, activation="tanh")),\n    layers.TimeDistributed(layers.Dense(N_TARGETS))])\n\nmodel.compile(\n    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0008),\n    loss=tf.keras.losses.Huber(delta=1.0), metrics=[tf.keras.metrics.MeanAbsoluteError(name="mae")])\n\nmodel.summary()\n\n'

In [18]:
# Visualizing the training and validation loss and MAE
"""
fig, ax = plt.subplots(figsize=(15, 8))
ax.plot(history.history['loss'], label='Training MSE', color='orange', linewidth=2, markersize=6)
ax.plot(history.history['val_loss'], label='Validation MSE', color='blue', linewidth=2, markersize=6)
ax.set_title('MSE (loss)')
ax.set_xlabel('Epoch')
ax.set_ylabel('MSE')
ax.legend()

fig, ax = plt.subplots(figsize=(15, 8))
ax.plot(history.history['mae'], label='Training MAE', color='orange', linewidth=2, markersize=6)
ax.plot(history.history['val_mae'], label='Validation MAE', color='blue', linewidth=2, markersize=6)
ax.set_title('MAE')
ax.set_xlabel('Epoch')
ax.set_ylabel('MAE')
ax.legend()
"""

"\nfig, ax = plt.subplots(figsize=(15, 8))\nax.plot(history.history['loss'], label='Training MSE', color='orange', linewidth=2, markersize=6)\nax.plot(history.history['val_loss'], label='Validation MSE', color='blue', linewidth=2, markersize=6)\nax.set_title('MSE (loss)')\nax.set_xlabel('Epoch')\nax.set_ylabel('MSE')\nax.legend()\n\nfig, ax = plt.subplots(figsize=(15, 8))\nax.plot(history.history['mae'], label='Training MAE', color='orange', linewidth=2, markersize=6)\nax.plot(history.history['val_mae'], label='Validation MAE', color='blue', linewidth=2, markersize=6)\nax.set_title('MAE')\nax.set_xlabel('Epoch')\nax.set_ylabel('MAE')\nax.legend()\n"

In [ ]:
# At first the past hours was 48 hours and next hours was 24 hours. and the splitting was 70% train, 15% val, 15% test.
# the train DS ended in Agust and test DS  was from November to December.
# The model could not see the cold season patterns.
# the zone 3 is challenging for the model, It has different behavior than zone 1 and 2.

#  I went for multi output model structure(with two commoon encoders and each zone has its own output)
# and loss weight that the model focus more on zone 3.
# Also I changed the split to 80% train and 20%  for val+test. The model can learn more patterns colder season
# What I noticed in Prepocessing notebook that power usage is lower in colder seasons because Moracco is a warm country.
# I picked the losses.LogCosh, because my DS is skewed, I mentioned that in preprocessing notebook.

# Zone 3 has now better results than before, but still has lower performance than other zones.
# R2 is 42%  for zone 3, that is  a lot better than before - 20 or -40. 
# Mape is almost 20% , which is the model predicts 1/5 wrongly.

# Zone1 and Zone2 have good results.
# I can see overfitting, the gap is big between train and val loss sepcially for zone1


In [20]:
# I used these sources:
# https://www.youtube.com/watch?v=kGdbPnMCdOg
# https://medium.com/data-science/step-by-step-understanding-lstm-autoencoder-layers-ffab055b6352